# 1. Data Augmentation (Add pattern variations)

In [9]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [10]:
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import TextVectorization, Bidirectional, LSTM, Attention, GlobalAveragePooling1D
from sklearn.preprocessing import LabelEncoder
import json
import pickle
import random
import re
import string
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aswan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Load dataset
with open('medicaldata.json') as file:
    data = json.load(file)

# Data Augmentation Functions
def augment_patterns(text):
    augmented = []
    words = text.split()
    
    # Synonym replacement
    for i in range(len(words)):
        synonyms = []
        for syn in wordnet.synsets(words[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name().replace('_', ' '))
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = words.copy()
            new_words[i] = synonym
            augmented.append(' '.join(new_words))
    
    # Question reformulation
    question_words = ['how', 'what', 'why', 'when', 'where']
    if any(text.lower().startswith(word) for word in question_words):
        augmented.append(text.replace("How to", "Best way to"))
        augmented.append(text.replace("What", "Can you explain what"))
        augmented.append(text + " please")
    
    return augmented

In [12]:
# Prepare original patterns and tags
patterns = []
tags = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Generate augmented data
augmented_patterns = []
augmented_tags = []
for pattern, tag in zip(patterns, tags):
    augmented = augment_patterns(pattern)
    augmented_patterns.extend(augmented)
    augmented_tags.extend([tag] * len(augmented))

# Combine with original data
augmented_patterns += patterns
augmented_tags += tags

In [13]:
# Text Preprocessing
def custom_standardization(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, f"[{re.escape(string.punctuation)}]", "")
    return text

vectorizer = TextVectorization(
    max_tokens=20000,
    output_mode='int',
    output_sequence_length=20,
    standardize=custom_standardization
)

# Adapt vectorizer
vectorizer.adapt(augmented_patterns)
vocab = vectorizer.get_vocabulary()


In [14]:
# Model Architecture
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = vectorizer(inputs)
x = layers.Embedding(input_dim=len(vocab), output_dim=128)(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)

# Corrected Attention Implementation
attention = layers.Attention()([x, x])  # Self-attention
x = layers.GlobalAveragePooling1D()(attention)

x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(len(np.unique(augmented_tags)), activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Label Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(augmented_tags)


In [15]:
# Dataset Preparation
dataset = tf.data.Dataset.from_tensor_slices(
    (augmented_patterns, encoded_labels)
).shuffle(len(augmented_patterns)) \
 .batch(32) \
 .prefetch(tf.data.AUTOTUNE)


In [56]:
# Compile with adjusted parameters
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),  # Increased learning rate
    metrics=['accuracy']
)

# Train with reduced complexity
history = model.fit(
    dataset,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)
    ]
)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0288 - loss: 3.5292 - learning_rate: 0.0010
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0617 - loss: 3.5167 - learning_rate: 0.0010
Epoch 3/50
 1/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0625 - loss: 3.4943

c:\Users\aswan\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
c:\Users\aswan\anaconda3\Lib\site-packages\keras\src\callbacks\callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0600 - loss: 3.5022 - learning_rate: 0.0010
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0907 - loss: 3.3841 - learning_rate: 0.0010
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1231 - loss: 3.0434 - learning_rate: 0.0010
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2228 - loss: 2.6468 - learning_rate: 0.0010
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2822 - loss: 2.3693 - learning_rate: 0.0010
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3462 - loss: 2.0658 - learning_rate: 0.0010
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4325 - loss: 1.7909 - learning_rate: 0.0010
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4929 - loss: 1.5848 - learning_rate: 0.0010
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6289 - loss: 1.2372 - learning_rate: 0.0010
Epoch 12/50
20/20 ━━━━━━━━━━━━━━━━━━━

In [57]:
# Save Model
model.save('new_medical_chatbot.h5')
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump({'config': vectorizer.get_config(),
                 'weights': vectorizer.get_weights()}, f)

In [ ]:
def get_response(text, confidence_threshold=0.6):
    # Preprocess
    text = text.lower().strip()
    
    # Convert to TensorFlow tensor
    input_tensor = tf.constant([text])  # Shape: (1,)
    
    # Predict
    prediction = model.predict(input_tensor, verbose=0)
    confidence = np.max(prediction)
    tag_index = np.argmax(prediction)
    tag = label_encoder.inverse_transform([tag_index])[0]
    
    # Handle low confidence
    if confidence < confidence_threshold:
        return "I'm not sure I understand. Could you please rephrase or provide more details?"
    
    # Get response
    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    
    return "I need to learn more about that. Please consult a medical professional."

# Chat Interface
print("Medical Chatbot: Hello! How can I assist you today? (type 'quit' to exit)")
while True:
    user_input = input("You: ").lower()
    if user_input.lower() == 'quit':
        break
    response = get_response(user_input)
    print(f"Chatbot: {response}")


Medical Chatbot: Hello! How can I assist you today? (type 'quit' to exit)
Chatbot: I'm not sure I understand. Could you please rephrase or provide more details?
Chatbot: I'm not sure I understand. Could you please rephrase or provide more details?
Chatbot: I'm not sure I understand. Could you please rephrase or provide more details?
Chatbot: Drink fluids, rest, and take paracetamol if needed. If the fever is high, consult a doctor.
Chatbot: I'm not sure I understand. Could you please rephrase or provide more details?
Chatbot: Common pregnancy symptoms include missed periods, nausea, and fatigue. Take a home pregnancy test and consult your doctor for confirmation.
Chatbot: Immunization helps protect against infectious diseases. Common vaccines include those for flu, measles, and polio.
Chatbot: Common pregnancy symptoms include missed periods, nausea, and fatigue. Take a home pregnancy test and consult your doctor for confirmation.
Chatbot: Asthma is a condition where the airways constr